# SIT742: Modern Data Science 
**(2021 Assessment Task 01 - Data Manipulation)**


# Part II - Web Crawling

## Overview
To better introduce all the professors including the emeritus professor, the professor and also associate professor in Deakin University School of IT, faculty will need to know all the citation information on all professors. Google Scholar is a web search engine that freely indexes the metadata of articles on many authors. Majority of the professors choose to use google scholar to track their publications and research works. Therefore,  the web crawling on google scholar will be able to have the citation information obtained across all the professors (who have the google scholar profile).


## 3. Professor List Generation

### 3.1 Import and install web crawling library. 

<div class="alert alert-block alert-warning">

**Code**: 
    Import and install your web crawling library here. 
</div> 

<div class="alert alert-block alert-warning">
    
**Report**: 
    **Part II 3.1** Please include your result for this coding task into your report Part II, with proper section title '3.1'.
</div>

*hint: you could use selenium by doing the pip install selenium, download the webdriver for chromedriver and define your webdriver for crawling. But you are free to use any other library*



In [25]:
!pip install -U selenium
!pip install webdriver-manager
!pip install urllib3
!pip3 install scholarly

import csv
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import urllib.request

from scholarly import scholarly
from urllib.request import urlopen

In [26]:
def parse_name(stringtext):
  return " ".join(stringtext.split(" ")[-2:])," ".join(stringtext.split(" ")[:-2])

In [27]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.deakin.edu.au/information-technology/staff-listing")



====== WebDriver manager ======
Current google-chrome version is 89.0.4389
Get LATEST driver version for 89.0.4389
Driver [C:\Users\ASUS\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


### 3.2 Find all professors in School of IT and save it as csv. 

<div class="alert alert-block alert-warning">

**Code**: 
    Find all professors in Schoolf of IT and save it as csv. The csv must have the full name of the professor (in one column), the title of the professor and University Name.
</div> 

<div class="alert alert-block alert-warning">
    
**Report**: 
    **Part II 3.2** Please include your result for this coding task into your report Part II, with proper section title '3.2'.
</div>



*hint: the url of the staff list for school of IT is https://www.deakin.edu.au/information-technology/staff-listing. you may need to iteratively find out html tag by table in the page to start crawl all the text from td (td is the elements in html table, check https://www.w3schools.com/html/html_tables.asp)*

In [28]:
Name_Prof = driver.find_elements_by_xpath('//td[@headers="table09355r1c1"]')

In [29]:
## Step 1:
# Create empty file of prof name without parse_name
with open('Prof_list0.csv', 'w') as f:
    for i in range(len(Name_Prof )):
        f.write(str(parse_name(Name_Prof[i].text)) + '\n')
        
f.close()

# Apply parse_name function

with open('Prof_list0.csv', 'r') as f:
    file_staff = open('Prof_list1.csv', 'w')
    for line in f:
        values = line.split(',')
        name = values[0].strip("('')")
        title = re.sub(r'\W+', ' ', values[1])
        file_staff.write(name + ' ' + title +'\n')
        
file_staff.close()        

# Save file

with open('Prof_list1.csv', 'r') as a:
    dataline = a.readlines()
prof_list = open('Prof_list2.csv','w')
for i in range(0,25):
    line = dataline[i].strip('\n')
    prof_list.write(line + ' ' + '\n')
prof_list.close()

In [30]:
## Step 2:
# Read file
prof_trial = pd.read_csv('Prof_list2.csv', 
                           names = ['First Name', 'Last Name', 'Title 1', 'Title 2'], 
                           delim_whitespace = True)

# Drop NA
prof_trial = prof_trial .replace(np.nan, '', regex=True)

# Format the table
prof_trial['Name'] = prof_trial[['First Name', 'Last Name']].apply(lambda x: ' '.join(x), axis=1)

prof_trial['Title'] = prof_trial[['Title 1', 'Title 2']].apply(lambda x: ' '.join(x), axis=1)

prof_only = prof_trial.drop(prof_trial.columns[[0, 1, 2, 3]], axis=1) 

prof_only['University'] = 'Deakin University'
prof_only

,Name,Title,University
0,Lynn Batten,Emeritus Professor,Deakin University
1,Andrzej Goscinski,Emeritus Professor,Deakin University
2,Jemal Abawajy,Professor,Deakin University
3,Maia Angelova,Professor,Deakin University
4,Gleb Beliakov,Professor,Deakin University
5,Terry Caelli,Professor,Deakin University
6,Jinho Choi,Professor,Deakin University
7,Chang-Tsun Li,Professor,Deakin University
8,Robin Doss,Professor,Deakin University
9,Peter Eklund,Professor,Deakin University


In [31]:
# Save to csv
prof_only.to_csv('Professor-list.csv', index=False)

## 4. Professor Citation Information Generation


### 4.1 Search the google scholar for all professors 

<div class="alert alert-block alert-warning">

**Code**: 
    Search the google scholar for all professors and obtain their citations-all, h-index-all, i10-all, citation-since2016,h-index-2016 and i10-since2016. Save the results as csv and must having **all the professors name, title and all 6 citation information.**
</div> 

<div class="alert alert-block alert-warning">
    
**Report**: 
    **Part II 4.1** Please include your result for this coding task into your report Part II, with proper section title '4.1'.
</div>


*hint: if the professors did not have the google scholar profle with the name, save the 6 citation information as string "na". you could use the while loop with condition or for loop with if condition or for loop with defined method having if condition to deal with those professors do not have google scholar*

In [33]:
# Search the google scholar
with open('Professor-citation0.csv', 'w') as f:
    for i in range(len(prof_only['Name'])):
        search_query = scholarly.search_author(prof_only['Name'][i])
        author = next(search_query, None)
        if author is None:
            f.write(prof_only['Name'][i] + ' '+ 'Na' + ' '+ 'Na' + ' '+ 'Na' + ' ' + 'Na' + ' ' + 'Na' + ' ' + 'Na' + '\n')
        else:
            x = scholarly.fill(author, sections=['basics', 'indices', 'coauthors'])
            name = prof_only['Name'][i]
            citedby =  str(author['citedby'])
            citedby5y = str(x['citedby5y'])
            hindex   =  str(x['hindex'])
            hindex5y   =     str(x['hindex5y'])
            i10index   =   str(x['i10index'])
            i10index5y    =    str(x['i10index5y'])
            
            f.write(name +' '+citedby+' '+citedby5y+' '+hindex+' '+hindex5y+' '+i10index+' '+i10index5y+'\n')
f.close()

In [35]:
# Read file 
prof_cit = pd.read_csv('Professor-citation0.csv', names =['First Name', 'Last Name', 
                                                            'citation-all', 'citation-since2016', 
                                                            'h-index-all', 'h-index-since2016', 
                                                            'i10-index-all', 'i10-index-since2016'], 
                                                            delim_whitespace = True)

# Remove First and Last Name
prof_cit = prof_cit.drop(prof_cit.columns[[0, 1]], axis=1) 

# Combine 2 dataframe prof_only and prof_cit
all_cit = pd.concat([prof_only,prof_cit], axis=1)

# Remove University
all_cit = all_cit.drop(all_cit.columns[[2]], axis=1) 

# Save to 'Professor-citation-information.csv'
all_cit.to_csv('Professor-citation-information.csv', index=False)

In [37]:
# Format table
prof_citation = all_cit.replace('Na', 0)

prof_citation[['citation-all', 'citation-since2016', 'h-index-all', 'h-index-since2016', 'i10-index-all', 
               'i10-index-since2016']]  = prof_citation[['citation-all', 'citation-since2016', 
                                                         'h-index-all', 'h-index-since2016', 
                                                         'i10-index-all', 'i10-index-since2016']].astype(int)
prof_citation

,Name,Title,citation-all,citation-since2016,h-index-all,h-index-since2016,i10-index-all,i10-index-since2016
0,Lynn Batten,Emeritus Professor,2535,1179,23,16,45,21
1,Andrzej Goscinski,Emeritus Professor,0,0,0,0,0,0
2,Jemal Abawajy,Professor,0,0,0,0,0,0
3,Maia Angelova,Professor,682,399,15,11,21,14
4,Gleb Beliakov,Professor,8182,4640,42,32,135,87
5,Terry Caelli,Professor,8669,1477,52,20,177,36
6,Jinho Choi,Professor,20442,7025,71,39,315,173
7,Chang-Tsun Li,Professor,4347,2341,33,21,104,65
8,Robin Doss,Professor,0,0,0,0,0,0
9,Peter Eklund,Professor,3968,783,34,14,72,23


### 4.2 Find out the professor name having the most citations 

<div class="alert alert-block alert-warning">

**Code**: 
    Find out the professor name having the most citations (please remove those professor who does not have google scholar page)
</div>

<div class="alert alert-block alert-warning">
    
**Report**: 
    **Part II 4.2** Please include your result for this coding task into your report Part II, with proper section title '4.2'.
</div>


In [40]:
# Professors have the most citations
prof_citation.loc[prof_citation['citation-all'].idxmax()]

Name                               Gang Li
Title                  Associate Professor
citation-all                         64452
citation-since2016                   30686
h-index-all                             75
h-index-since2016                       60
i10-index-all                          144
i10-index-since2016                    130
Name: 20, dtype: object

### 4.3 Find out the row for associate professor having the most i10-index since 2016 
<div class="alert alert-block alert-warning">

**Code**: 
    Find out the row for associate professor having the most i10-index since 2016 (please remove those professor who does not have google scholar page)
</div>


<div class="alert alert-block alert-warning">
    
**Report**: 
    **Part II 4.3** Please include your result for this coding task into your report Part II, with proper section title '4.3'.
</div>


In [42]:
# Associate professors have the most i10_index since 2016
prof_citation.nlargest(1, 'i10-index-since2016')

,Name,Title,citation-all,citation-since2016,h-index-all,h-index-since2016,i10-index-all,i10-index-since2016
6,Jinho Choi,Professor,20442,7025,71,39,315,173


### 4.4 Find out all the professors name who has the citations_since2016 > 2500
<div class="alert alert-block alert-warning">

**Code**: 
    Find out the row for associate professor having the most i10-index since 2016 (please remove those professor who does not have google scholar page)
</div>

<div class="alert alert-block alert-warning">
    
**Report**: 
    **Part II 4.4** Please include your result for this coding task into your report Part II, with proper section title '4.4'.
</div>


In [45]:
# Professors have the citations_since2016 > 2500
prof_citation.loc[(prof_citation['citation-since2016'] > 2500)]

,Name,Title,citation-all,citation-since2016,h-index-all,h-index-since2016,i10-index-all,i10-index-since2016
4,Gleb Beliakov,Professor,8182,4640,42,32,135,87
6,Jinho Choi,Professor,20442,7025,71,39,315,173
10,Seng Loke,Professor,7188,2913,38,21,126,50
13,Yong Xiang,Professor,13627,6573,57,38,161,121
15,Arkady Zaslavsky,Professor,14966,8361,50,33,177,80
17,Andrew Cain,Associate Professor,3195,2935,17,16,23,21
20,Gang Li,Associate Professor,64452,30686,75,60,144,130
21,Jianxin Li,Associate Professor,6024,4328,43,34,117,105
22,Xiao Liu,Associate Professor,24444,18033,36,35,53,52
